Text Summarization with Hugging Face
- https://huggingface.co/transformers/v3.3.1/task_summary.html

In [1]:
print("Welcome")

Welcome


In [2]:
!nvidia-smi

Tue Mar 12 07:23:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              27W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install transformers[sentencepiece]     # convert data to tokens


In [4]:
!pip install sacrebleu rouge-score    # evaluation metrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 6.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=eb704eb22117e810225bb11150e63fdfadab9777ec5cbc49a5ec55b22e09c054
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


SacreBLEU : metric commonly used for evaluating quality of machine generated translation, typically in the context of machine translation tasks. It is often used in NLP competitions and research to assess the performance of translation models.

Rouge (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used for evaluating the quality of summaries by comparing them to reference summaries. Rouge metrics are commonly used in NLP tasks, particularly in the evaluation of text summarization systems.

In [5]:
!pip install py7zr datasets -q

In [6]:
!pip install --upgrade accelerate    # for distributed process of cpu/gpu/f16
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 80.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 16.2 MB/s eta 0:00:00


https://huggingface.co/docs/accelerate/index

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm

nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


2024-03-12 07:24:40.059247: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 07:24:40.059355: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 07:24:40.193188: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
from datasets import load_dataset, load_from_disk, load_metric

In [10]:
import torch

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
device

'cuda'

Hugging Face's Pegasus model is the model developed specifically for text summarization task
- https://huggingface.co/google/pegasus-large
- Pegasus : Pre-training with Extracted Gap Sentences for Abstractive Summarization

In [13]:
model = "google/pegasus-cnn_dailymail"

In [14]:
# Get embeddings

tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [15]:
# Load the model to device for Seq generation
model = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Training of already trained model is Finetuning
- We are are taking pretrined model, which does not have the exact behaviour of the which we want to do.
- For such cases:
1. We can finetune the model, do some augumentations
2. Maintain some knoweldge base to extract response as per query (RAG)

https://huggingface.co/docs/datasets/load_hub
- Supervised Finetuning : where we are training our data with the pretrained models

In [16]:
dataset = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [18]:
dataset['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [19]:
dataset['train']['dialogue'][3]

"Edward: Rachel, I think I'm in ove with Bella..\r\nrachel: Dont say anything else..\r\nEdward: What do you mean??\r\nrachel: Open your fu**ing door.. I'm outside"

In [20]:
dataset['train']['summary'][3]


'Edward thinks he is in love with Bella. Rachel wants Edward to open his door. Rachel is outside. '

In [21]:
for split in dataset:
  print(split)

train
test
validation


In [22]:
for split in dataset:
  print(dataset[split])


Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})


In [23]:
split_lengths = [len(dataset[split]) for split in dataset]

print(split_lengths)

[14732, 819, 818]


In [24]:
dataset['train'].column_names

['id', 'dialogue', 'summary']

In [25]:
dataset['test']['dialogue'][2]

"Lenny: Babe, can you help me with something?\r\nBob: Sure, what's up?\r\nLenny: Which one should I pick?\r\nBob: Send me photos\r\nLenny:  <file_photo>\r\nLenny:  <file_photo>\r\nLenny:  <file_photo>\r\nBob: I like the first ones best\r\nLenny: But I already have purple trousers. Does it make sense to have two pairs?\r\nBob: I have four black pairs :D :D\r\nLenny: yeah, but shouldn't I pick a different color?\r\nBob: what matters is what you'll give you the most outfit options\r\nLenny: So I guess I'll buy the first or the third pair then\r\nBob: Pick the best quality then\r\nLenny: ur right, thx\r\nBob: no prob :)"

In [26]:
dataset['test']['summary'][2]

"Lenny can't decide which trousers to buy. Bob advised Lenny on that topic. Lenny goes with Bob's advice to pick the trousers that are of best quality."

Tokenization of the dataset

In [27]:
def convert_examples_to_features(data_in_batch):
    input_encoding =tokenizer(data_in_batch['dialogue'], max_length=1024, truncation=True)
    
    target_encoding = tokenizer(data_in_batch['summary'], max_length=128, truncation=True)
    
    return {
        "input_ids":input_encoding['input_ids'],
        "attention_mask":input_encoding["attention_mask"],
        "labels": target_encoding["input_ids"]
    }

To apply the preprocessing function over the entire dataset, use 🤗 Datasets map method
- https://huggingface.co/docs/transformers/tasks/summarization

In [28]:
dataset_enc = dataset.map(convert_examples_to_features, batched = True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Observe : 'input_ids', 'attention_mask', 'labels' are added to the dataset

In [29]:
dataset_enc['train'][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.',
 'input_ids': [12195,
  151,
  125,
  7091,
  3659,
  107,
  842,
  119,
  245,
  181,
  152,
  10508,
  151,
  7435,
  147,
  12195,
  151,
  125,
  131,
  267,
  650,
  119,
  3469,
  29344,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [12195, 7091, 3659, 111, 138, 650, 10508, 181, 3469, 107, 1]}

In [30]:
dataset_enc['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

Attention_Mask : 
- In context of NLP and transformers based models, an attention mask is a binary mask that indicates which tokens in a sequence should be attented to and which should be ignored.
- The mask is used during self attention mechanism to control the flow of information between different positions in the input sequence.
- The attention mask is typically a tensor with the same length as the input sequence, where each element is either 1 or 0

The Finetuning

In [31]:
from transformers import TrainingArguments, Trainer

In [32]:
# training_args = TrainingArguments(
#     output_dir = 'pegasus-samsum', num_train_epochs=2, warmup_steps=500,
#     per_device_train_batch_size=4, per_device_eval_batch_size=4,
#     weight_decay=0.05, logging_steps=10,
#     evaluation_strategy='steps', eval_steps = 500, save_steps= 1e6,
#     gradient_accumulation_steps = 50
    
# )

- Trying to  resolve OutOfMemory Error with https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments

In [33]:
training_args = TrainingArguments(
    output_dir = 'pegasus-samsum', 
    evaluation_strategy = "steps",
    num_train_epochs=15, 
    warmup_steps=500,
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    weight_decay=0.01, 
    logging_steps=50,
#     eval_steps = 500, 
    save_total_limit=2,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    
)

##Number of predictions steps to accumulate the output tensors for, before moving the results to the CPU. If left unset, the whole predictions are accumulated on GPU/NPU/TPU before being moved to the CPU (faster but requires more memory).


- https://huggingface.co/docs/transformers/main_classes/trainer

In [34]:
# Create Data Collator
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The primary purposes of a data collator are:

Batch Creation: It assembles individual data samples into batches of suitable sizes. Batching is a common technique used during model training to improve computational efficiency by processing multiple examples simultaneously.

Padding: It handles sequences of varying lengths by padding or truncating them to a fixed length within each batch. This ensures that all sequences within a batch have the same length, which is required for efficient parallel computation on modern hardware (e.g., GPUs).

Tokenization: If necessary, it tokenizes raw text inputs into numerical representations suitable for consumption by the model. This process typically involves converting words or characters into numerical tokens and possibly applying additional preprocessing steps such as lowercasing or special token insertion.

Masking: For tasks like language modeling or masked language modeling, where certain tokens are masked or replaced with special tokens during training, the data collator may perform the masking operation on the input data.

Special Handling: It may handle special cases or data formats specific to the task or model being used. For example, in tasks involving multiple input modalities (e.g., text and images), the data collator may need to process and combine data from different sources.

In [35]:
trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator = seq2seq_data_collator,
                 train_dataset = dataset_enc['test'],    #as its a huge dataset, we going to train our model on test dataset to save time and memory
                 eval_dataset = dataset_enc['validation'])

In [36]:
!pip install wandb --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.16.3
    Uninstalling wandb-0.16.3:
      Successfully uninstalled wandb-0.16.3


In [37]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
50,3.022600,2.336237
100,2.627500,1.962225
150,2.148100,1.770909
200,1.931300,1.675590
250,1.775500,1.622203
300,1.626100,1.589017
350,1.548500,1.567199
400,1.472700,1.555128
450,1.363600,1.555890
500,1.289400,1.549665


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=765, training_loss=1.5979792887868445, metrics={'train_runtime': 2046.1561, 'train_samples_per_second': 6.004, 'train_steps_per_second': 0.374, 'total_flos': 4692145956077568.0, 'train_loss': 1.5979792887868445, 'epoch': 14.95})

Training Loss Reduction: The training loss consistently decreases as the number of training steps increases. This indicates that the model is learning from the training data and improving its performance over time. This is expected behavior in a well-performing training process.

Validation Loss Trend: Initially, the validation loss decreases along with the training loss, suggesting that the model is generalizing well to unseen data. However, after a certain point (around step 400), the validation loss starts to stabilize and even slightly increase. This indicates that the model might be overfitting to the training data as the training progresses beyond this point.

Model Performance: The model achieves a relatively low training loss, indicating that it is able to fit the training data well. However, the validation loss, although decreasing initially, stabilizes and then slightly increases, indicating that the model's performance on unseen validation data might not improve significantly beyond a certain point. This suggests that the model may have reached its capacity to generalize from the training data.

Possible Actions: To address overfitting and improve generalization, techniques such as regularization, early stopping, or adjusting model complexity can be employed. Additionally, monitoring the validation loss and performance on separate validation datasets can help in identifying the optimal point to stop training and prevent overfitting.



Training Loss: The final training loss is approximately 1.598, indicating that the model achieved a relatively low loss on the training data. This suggests that the model has learned to fit the training data to some extent.

Training Runtime: The total training runtime is approximately 2046.1561 seconds, with an average of around 6.004 samples processed per second. This information gives insight into the computational resources required for training the model.

Training Steps and Epochs: The training process has iterated over 765 global steps, which corresponds to approximately 14.95 epochs. This indicates that the model has been trained on the entire training dataset multiple times (approximately 15 times in this case).

Training Performance: The reported metrics suggest that the training process has been relatively stable and efficient, with an average of around 0.374 steps processed per second. Additionally, the total number of floating-point operations (FLOPs) performed during training is reported as 4.692e+15, providing an estimate of the computational workload.

Overall, the provided training output indicates that the model has undergone a successful training process, achieving a reasonably low training loss within a reasonable training runtime. However, further evaluation on validation or test data would be necessary to assess the model's generalization performance and determine its suitability for deployment in practical applications.

Lets Evaluate the Model

In [38]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]
        
def calculate_metric_on_test_data(dataset, metric, model, tokenizer,
                                 batch_size = 8, device = device,
                                 column_text = "article",
                                 column_summary = 'highlights'):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))
    
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                          padding='max_length', return_tensors='pt')
        
        summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                                  attention_mask = inputs['attention_mask'].to(device),
                                  length_penalty = 0.8, num_beams = 8, max_length=128)
        
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                             clean_up_tokenization_spaces=True)
                                for s in summaries]
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        metric.add_batch(predictions = decoded_summaries, references = target_batch)
        
    # Finally compute and return the ROUGE scores
    score = metric.compute()
    return score

- Rouge-1 measures the overlap of unigrams between the generated summary and the reference summary.
- Rouge-2 measures the overlap of bigrams.
- Rouge-L measures the longest common subsequence between the generated and reference summaries.
- Rouge-Lsum is an F1 score variant of Rouge-L.

In [39]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_metric = load_metric('rouge')

In [40]:
score = calculate_metric_on_test_data(
    dataset['train'][0:10], rouge_metric, trainer.model, tokenizer, batch_size=2, column_text='dialogue', column_summary='summary')

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame(rouge_dict, index = [f'pegasus'])

100%|██████████| 5/5 [00:06<00:00,  1.29s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.024261,0.0,0.02395,0.024125


- Rouge-1 and Rouge-L Scores: Both Rouge-1 and Rouge-L scores are positive, indicating some level of overlap between the generated summary and the reference summary in terms of unigrams and longest common subsequences. This suggests that the Pegasus model is capturing important content from the reference summaries.

- Rouge-2 Score: The Rouge-2 score is 0.0, which means there's no overlap between bigrams in the generated and reference summaries. This suggests that the model may struggle to generate accurate sequences of two consecutive words.

- Rouge-Lsum Score: Rouge-Lsum score is also positive, indicating some level of agreement between the generated and reference summaries. This score suggests that while the model may not precisely match the reference summaries in all aspects, it still captures important content.

- Overall, these scores suggest that the Pegasus model performs reasonably well in terms of capturing unigrams and longest common subsequences from the reference summaries but may struggle with capturing bigrams accurately. Further analysis and comparison with other models may provide additional insights into its performance.

Save the pretrained model

In [41]:
import os
print(os.getcwd())

/kaggle/working


In [42]:
model.save_pretrained('pegasus_samsum_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [43]:
tokenizer.save_pretrained('samsum-tokenizer')

('samsum-tokenizer/tokenizer_config.json',
 'samsum-tokenizer/special_tokens_map.json',
 'samsum-tokenizer/spiece.model',
 'samsum-tokenizer/added_tokens.json',
 'samsum-tokenizer/tokenizer.json')

Load the saved model and tokenizer

In [44]:
tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/samsum-tokenizer')

In [45]:
finetuned_model_pegasus = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/pegasus_samsum_model').to(device)

In [46]:
finetuned_model_pegasus

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

Prediction / Inference

In [47]:
# we have finetuned our model on 'test' data
# will use some data from 'train' to prediction

sample_text = dataset['train']['dialogue'][2]
sample_text

"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like using post-its in kaban style"

In [48]:
sample_text_summary = dataset['train']['summary'][2]
sample_text_summary

'Kim may try the pomodoro technique recommended by Tim to get more stuff done.'

Lets try to predict this summary with finetuned model and Hugging face Pipeline
- https://huggingface.co/docs/transformers/en/main_classes/pipelines

In [49]:
pipeline_object = pipeline('summarization', model = finetuned_model_pegasus, tokenizer = tokenizer)

In [50]:
pipeline_object(sample_text)

[{'summary_text': 'Kim was going to do lots of stuff but ended up procrastinating. She will move her ass tomorrow and do everything. Tim recommends Pomodoro technique for doing chores.'}]

In [51]:
predicted_summary = pipeline_object(sample_text)[0]["summary_text"]
predicted_summary

'Kim was going to do lots of stuff but ended up procrastinating. She will move her ass tomorrow and do everything. Tim recommends Pomodoro technique for doing chores.'

To make the summary prediction task more better, we can add extra keyword arguments 

In [52]:
gen_kwargs = {'length_penalty':0.08, 'num_beams':8, 'max_length':128}

In [53]:
# pass this dict args to pipeline object
pipeline_object(sample_text, **gen_kwargs)

[{'summary_text': 'Kim was going to do lots of stuff but ended up procrastinating. She will move her ass tomorrow and do everything. Tim recommends Pomodoro technique for doing chores.'}]

In [54]:
predicted_summary = pipeline_object(sample_text)[0]["summary_text"]
predicted_summary

'Kim was going to do lots of stuff but ended up procrastinating. She will move her ass tomorrow and do everything. Tim recommends Pomodoro technique for doing chores.'

Lets compare Actuall summary and predicted summary

In [55]:
sample_text = dataset["train"][0]["dialogue"]

reference = dataset["train"][0]["summary"]

pipe = pipeline("summarization", model=finetuned_model_pegasus,tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


Dialogue:
Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

Reference Summary:
Amanda baked cookies and will bring Jerry some tomorrow.

Model Summary:
Amanda baked cookies. She'll bring them tomorrow. Jerry will get them. Amanda is going to bring them to Jerry's house in the morning.
